In [1]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import joblib

In [2]:
df=pd.read_csv("03_app_encoded.csv")

In [3]:
df.head()

,sk_id_curr,target,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,region_population_relative,days_birth,days_employed,...,organization_type_Transport: type 1,organization_type_Transport: type 2,organization_type_Transport: type 3,organization_type_Transport: type 4,organization_type_University,organization_type_XNA,prev_name_contract_type_Consumer loans,prev_name_contract_type_Revolving loans,prev_name_contract_type_Unknown,prev_name_contract_type_XNA
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,9461,637,...,False,False,False,False,False,False,True,False,False,False
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,16765,1188,...,False,False,False,False,False,False,True,False,False,False
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,19046,225,...,False,False,False,False,False,False,True,False,False,False
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,19005,3039,...,False,False,False,False,False,False,False,False,False,False
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,19932,3038,...,False,False,False,False,False,False,False,False,False,False


In [4]:
X = df.drop(columns=['target','sk_id_curr'])
y = df['target']


créer un data_template.csv contenant les moyennes de toutes les colonnes

et surtout on ne veut ue une ligne Tu charges 1 ligne avec la moyenne de ces colonnes et l'utilisateur modifieras les valeurs (donc les valeurs ue tauras mit dans le formulaire)

In [5]:
# ensuite on veut calculer la moyenne de chaque colonne
template = X.mean(numeric_only=True).to_frame().T


In [6]:
template.shape

(1, 203)

In [7]:
template.to_csv("data_template.csv", index=False)

partie entrainement: tout dabord on va charger tout ce dont on a besoin:


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [9]:
import re

# Nettoyage global pour X, X_train et X_test
clean_columns = [re.sub(r"[^A-Za-z0-9_]", "_", col) for col in X_train.columns]

X_train.columns = clean_columns
X_test.columns = clean_columns


In [10]:
model= LGBMClassifier(class_weight='balanced', random_state=42, n_jobs=-1)
model.fit(X_train,y_train)
model.score(X_test,y_test)


[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077099 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9624
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 192
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


0.7174121587564835

In [11]:
cv=cv=StratifiedKFold(5,shuffle=True,random_state=42)
cross_val_score(model,X_train,y_train,cv=cv,scoring='recall')

[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.100846 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9630
[LightGBM] [Info] Number of data points in the train set: 196806, number of used features: 191
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 15888, number of negative: 180918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9624
[LightGBM] [Info] Number of data points in the train set: 196806, number of used features: 192
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

array([0.67547835, 0.68680765, 0.6805136 , 0.67925478, 0.67900302])

In [12]:
lgbm = LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    n_jobs=-1,
    random_state=42
)

# 2. Grille des hyperparamètres
param_grid = {
    'num_leaves': [31],
    'max_depth': [5, -1],
    'learning_rate': [0.05],
    'n_estimators': [100,300],
    'scale_pos_weight': [5, 8]  # pour les classes déséquilibrées
}



# 4. GridSearchCV avec scoring orienté F1 pour prendre en compte les faux négatifs
grid = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='recall',
    cv=cv,
    verbose=2,
    n_jobs=-1
)



In [13]:
grid.fit(X_train, y_train)
model_final=grid.best_estimator_

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.070522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9624
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 192
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482


In [14]:
y_pred=model_final.predict(X_test)
print(confusion_matrix(y_test,y_pred))

[[45915 10623]
 [ 2135  2830]]


In [15]:
joblib.dump(model_final,"model_final_203features.pkl")

['model_final_203features.pkl']